# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [2]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform
import numpy as np

In [3]:
data = pd.read_csv('customer_product_sales.csv')

In [4]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [5]:
data1 = data.groupby(['CustomerID','ProductName']).sum()

In [6]:
data1

SalesID  ProductID  Quantity
CustomerID ProductName                                                
33         Apricots - Dried                172592        324         1
           Assorted Desserts              5363705        219         1
           Bandage - Flexible Neon        6299165        273         1
           Bar Mix - Pina Colada, 355 Ml  3147536        281         1
           Beans - Kidney, Canned         6507033        300         1
...                                           ...        ...       ...
98200      Vol Au Vents                   5160346        272        50
           Wasabi Powder                  4438427         80        25
           Wine - Fume Blanc Fetzer       6226060         76        25
           Wine - Hardys Bankside Shiraz   266676        360        25
           Yogurt - French Vanilla        2112061        320        25

[63628 rows x 3 columns]

# Step 2: Use the pivot_table method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the fillna method.

In [7]:
data2 = data_pivot = pd.pivot_table(data1, 
                       index="ProductName",
                       values="Quantity", 
                       columns="CustomerID",
                      fill_value=0)

In [8]:
data2

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,25,0,0,0,0,0,0
Yoghurt Tubes,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,25,0,0
"Yogurt - Blueberry, 175 Gr",0,1,0,0,0,0,0,0,0,0,...,25,0,0,25,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [9]:
from scipy.spatial.distance import pdist, squareform

distances = pdist(data2.T, 'euclidean')
print(distances)
print("-----Squareform-----")
print(squareform(distances))

[ 11.91637529  10.48808848  11.22497216 ... 304.13812651 305.16389039
 303.10889132]
-----Squareform-----
[[  0.          11.91637529  10.48808848 ... 228.62851966 239.
  229.77380181]
 [ 11.91637529   0.          11.74734012 ... 228.01096465 239.03765394
  229.70415756]
 [ 10.48808848  11.74734012   0.         ... 228.08112592 238.26665734
  229.77380181]
 ...
 [228.62851966 228.01096465 228.08112592 ...   0.         304.13812651
  305.16389039]
 [239.         239.03765394 238.26665734 ... 304.13812651   0.
  303.10889132]
 [229.77380181 229.70415756 229.77380181 ... 305.16389039 303.10889132
    0.        ]]


In [10]:
distances = pd.DataFrame(1/(1 + squareform(pdist(data2.T, 'euclidean'))), 
                         index=data2.columns, columns=data2.columns)

distances

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.004515,0.004521,0.004543,0.004526,0.004530,0.004533,0.004534,0.004526,0.004528,0.004523,...,0.003509,0.003509,0.003300,0.003653,0.003579,1.000000,0.003638,0.003418,0.003255,0.003245
98069,0.004583,0.004614,0.004595,0.004578,0.004578,0.004578,0.004589,0.004581,0.004585,0.004592,...,0.003509,0.003509,0.003369,0.003509,0.003496,0.003638,1.000000,0.003469,0.003345,0.003381
98159,0.004355,0.004367,0.004365,0.004365,0.004367,0.004363,0.004366,0.004363,0.004369,0.004365,...,0.003509,0.003565,0.003322,0.003431,0.003322,0.003418,0.003469,1.000000,0.003277,0.003266


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [11]:
similarities = distances[33].sort_values(ascending=False)[1:6]
similarities
top5 = pd.DataFrame(similarities)
top5= top5.index.tolist()

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [12]:
data3 = data1.loc[top5]
data3

SalesID  ProductID  Quantity
CustomerID ProductName                                                
264        Apricots - Halves              3730301        278         1
           Apricots Fresh                 5035449        158         1
           Bacardi Breezer - Tropical      756802         49         1
           Bagel - Plain                  6728381        262         1
           Banana - Leaves                6298459        271         1
...                                           ...        ...       ...
3305       Wine - Ej Gallo Sierra Valley   464102        884         2
           Wine - Pinot Noir Latour       1308025         87         1
           Wine - Redchard Merritt         535578        425         1
           Wine - White Cab Sauv.on        238590        380         1
           Yogurt - French Vanilla        6757819        320         1

[270 rows x 3 columns]

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [13]:
data4 = data3.groupby(['ProductName']).sum(['Quantity']).sort_values(ascending=False, by=['Quantity'])
data4

,SalesID,ProductID,Quantity
ProductName,,,
Butter - Unsalted,13554503,1206,3
Wine - Ej Gallo Sierra Valley,5892505,1326,3
Towels - Paper / Kraft,5618699,957,3
Soup - Campbells Bean Medley,9218997,1146,3
Wine - Blue Nun Qualitatswein,9241577,807,3
...,...,...,...
Hinge W Undercut,6577594,297,1
Ice Cream Bar - Hageen Daz To,5840790,246,1
Jagermeister,5687769,374,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [14]:
data33 = pd.DataFrame(data2[33])
merged_set = pd.merge(data4, data33, on='ProductName')
merged_sorted = merged_set.sort_values([33,'Quantity'], ascending=[True,False])
top5merged = merged_sorted.head(5)
top5merged


,SalesID,ProductID,Quantity,33
ProductName,,,,
Butter - Unsalted,13554503,1206,3,0
Wine - Ej Gallo Sierra Valley,5892505,1326,3,0
Soup - Campbells Bean Medley,9218997,1146,3,0
Wine - Blue Nun Qualitatswein,9241577,807,3,0
Chicken - Soup Base,5301230,258,2,0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [15]:
unique_customerids = data['CustomerID'].unique().tolist()
unique_customerids

    


[61288,
 77352,
 40094,
 23548,
 78981,
 83106,
 11253,
 35107,
 15088,
 26031,
 86595,
 46715,
 91013,
 69931,
 19753,
 9546,
 10224,
 72846,
 28976,
 3267,
 19260,
 76530,
 30196,
 48818,
 15166,
 11443,
 79060,
 82597,
 62411,
 42611,
 45420,
 83495,
 47122,
 30815,
 53198,
 18591,
 62034,
 1428,
 2187,
 55162,
 22345,
 81876,
 48641,
 81034,
 29681,
 87947,
 28867,
 69967,
 94420,
 83158,
 84364,
 44742,
 7772,
 47303,
 75791,
 77756,
 97324,
 13912,
 52700,
 21734,
 6817,
 21095,
 28875,
 89588,
 1034,
 54827,
 29287,
 11102,
 52155,
 50748,
 88552,
 80938,
 60481,
 92168,
 66232,
 64348,
 37510,
 44408,
 89254,
 477,
 51483,
 97201,
 42157,
 52091,
 86065,
 18027,
 93882,
 42303,
 32010,
 53997,
 63033,
 76475,
 98185,
 84779,
 82146,
 54495,
 3903,
 79854,
 25449,
 85496,
 95017,
 33759,
 43652,
 35873,
 97769,
 891,
 85161,
 25327,
 25624,
 68416,
 44090,
 71595,
 68336,
 38497,
 10016,
 36751,
 58421,
 45498,
 50774,
 79997,
 45313,
 55715,
 40901,
 1577,
 3909,
 18480,
 41405

In [20]:
recommendations = {}
unique_customerids = list(data['CustomerID'])
for i in unique_customerids[1:6]:
    similarities = distances[i].sort_values(ascending=False)[1:6]
    top_5 = pd.DataFrame(similarities)
    top_5= top_5.index.tolist()
    df = data1.loc[top_5]
    df1 = df.groupby(['ProductName']).sum(['Quantity']).sort_values(ascending=False, by=['Quantity'])
    df2 = pd.DataFrame(data2[i])
    merged_newset = pd.merge(df1, df2, on='ProductName')
    merged_newsorted = merged_newset.sort_values([i,'Quantity'], ascending=[True,False])
    top5merged = merged_newsorted.head(5)
    recommendations.update(top5merged)

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [52]:
final = pd.DataFrame(recommendations).T
final


,Bagel - Plain,Baking Powder,Beef - Inside Round,Bouq All Italian - Primerba,Bread - French Baquette,"Bread - Roll, Soft White Round",Bread - Rye,Cake - Mini Cheesecake,Cheese - Cottage Cheese,Cinnamon Buns Sticky,...,Olives - Stuffed,Onions - Vidalia,Pastry - Choclate Baked,Pork - Kidney,Quiche Assorted,Scallops - Live In Shell,"Soup - Canadian Pea, Dry Mix",Truffle Cups - Brown,Veal - Sweetbread,Wine - Blue Nun Qualitatswein
SalesID,NaN,NaN,NaN,9328537.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,11275187.0,8399094.0,NaN,NaN,14045956.0,7449926.0,NaN,NaN,NaN
ProductID,NaN,NaN,NaN,996.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,966.0,890.0,NaN,NaN,1278.0,762.0,NaN,NaN,NaN
Quantity,NaN,NaN,NaN,67.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,62.0,48.0,NaN,NaN,50.0,46.0,NaN,NaN,NaN
77352,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
40094,NaN,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
23548,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
78981,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0
83106,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [36]:
distances = pd.DataFrame(1/(1 + squareform(pdist(data2.T, 'cosine'))), 
                         index=data2.columns, columns=data2.columns)

In [37]:
similarities = distances[33].sort_values(ascending=False)[1:6]
similarities
top5 = pd.DataFrame(similarities)
top5= top5.index.tolist()

In [38]:
data3 = data1.loc[top5]

In [39]:
data4 = data3.groupby(['ProductName']).sum(['Quantity']).sort_values(ascending=False, by=['Quantity'])

In [40]:
data33 = pd.DataFrame(data2[33])
merged_set = pd.merge(data4, data33, on='ProductName')
merged_sorted = merged_set.sort_values([33,'Quantity'], ascending=[True,False])
top5merged = merged_sorted.head(5)
top5merged

,SalesID,ProductID,Quantity,33
ProductName,,,,
Knife Plastic - White,11486267,835,64,0
"Soup - Campbells, Beef Barley",9214355,750,52,0
Onions - Cippolini,14851940,9,48,0
Tea - Herbal Sweet Dreams,12558319,564,45,0
Muffin - Zero Transfat,5222718,218,40,0


In [41]:
unique_customerids2 = data['CustomerID'].unique().tolist()
unique_customerids2



[61288,
 77352,
 40094,
 23548,
 78981,
 83106,
 11253,
 35107,
 15088,
 26031,
 86595,
 46715,
 91013,
 69931,
 19753,
 9546,
 10224,
 72846,
 28976,
 3267,
 19260,
 76530,
 30196,
 48818,
 15166,
 11443,
 79060,
 82597,
 62411,
 42611,
 45420,
 83495,
 47122,
 30815,
 53198,
 18591,
 62034,
 1428,
 2187,
 55162,
 22345,
 81876,
 48641,
 81034,
 29681,
 87947,
 28867,
 69967,
 94420,
 83158,
 84364,
 44742,
 7772,
 47303,
 75791,
 77756,
 97324,
 13912,
 52700,
 21734,
 6817,
 21095,
 28875,
 89588,
 1034,
 54827,
 29287,
 11102,
 52155,
 50748,
 88552,
 80938,
 60481,
 92168,
 66232,
 64348,
 37510,
 44408,
 89254,
 477,
 51483,
 97201,
 42157,
 52091,
 86065,
 18027,
 93882,
 42303,
 32010,
 53997,
 63033,
 76475,
 98185,
 84779,
 82146,
 54495,
 3903,
 79854,
 25449,
 85496,
 95017,
 33759,
 43652,
 35873,
 97769,
 891,
 85161,
 25327,
 25624,
 68416,
 44090,
 71595,
 68336,
 38497,
 10016,
 36751,
 58421,
 45498,
 50774,
 79997,
 45313,
 55715,
 40901,
 1577,
 3909,
 18480,
 41405

In [44]:
recommendations2 = {}
unique_customerids2 = list(data['CustomerID'])
for i in unique_customerids2[1:6]:
    similarities = distances[i].sort_values(ascending=False)[1:6]
    top_5 = pd.DataFrame(similarities)
    top_5= top_5.index.tolist()
    df = data1.loc[top_5]
    df1 = df.groupby(['ProductName']).sum(['Quantity']).sort_values(ascending=False, by=['Quantity'])
    df2 = pd.DataFrame(data2[i])
    merged_newset = pd.merge(df1, df2, on='ProductName')
    merged_newsorted = merged_newset.sort_values([i,'Quantity'], ascending=[True,False])
    top5merged = merged_newsorted.head(5)
    recommendations2.update(top5merged)

In [46]:
final2 = pd.DataFrame(recommendations2)

In [47]:
final2

,SalesID,ProductID,Quantity,77352,40094,23548,78981,83106
Bagel - Plain,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
Baking Powder,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
Beef - Inside Round,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
Bouq All Italian - Primerba,9328537.0,996.0,67.0,NaN,NaN,NaN,NaN,0.0
Bread - French Baquette,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
"Bread - Roll, Soft White Round",NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
Bread - Rye,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
Cake - Mini Cheesecake,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
Cheese - Cottage Cheese,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
Cinnamon Buns Sticky,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
